In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('cars.csv')
df.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.5,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.2,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.8,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.6,Nigerian Used,"125,206"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7184 non-null   object 
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7188 non-null   float64
 7   Type                    7008 non-null   object 
 8   Distance                4845 non-null   object 
dtypes: float64(1), object(8)
memory usage: 506.7+ KB


In [12]:
df.shape

(7205, 9)

In [13]:
df.duplicated().sum()

0

In [14]:
df.isnull().sum()

VehicleID                    0
Location                     0
Maker                        0
Model                        0
Year                        21
Colour                       0
Amount (Million Naira)      17
Type                       197
Distance                  2360
dtype: int64

In [20]:
df['Year'] = df['Year'].str.replace(',', '')
df['Distance'] = df['Distance'].str.replace(',', '')

In [29]:
df['Distance'] = df['Distance'].fillna(df['Distance'].median())
df['Year'] = df['Year'].fillna(df['Year'].median())
df['Type'] = df['Type'].fillna(method = 'ffill')
df['Amount (Million Naira)'] = df['Amount (Million Naira)'].fillna(df['Amount (Million Naira)'].mean())

In [30]:
df.isnull().sum()

VehicleID                 0
Location                  0
Maker                     0
Model                     0
Year                      0
Colour                    0
Amount (Million Naira)    0
Type                      0
Distance                  0
dtype: int64

In [31]:
df['Year'] = df['Year'].astype(int)
df['Distance'] = df['Distance'].astype(float)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7205 non-null   int32  
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7205 non-null   float64
 7   Type                    7205 non-null   object 
 8   Distance                7205 non-null   float64
dtypes: float64(2), int32(1), object(6)
memory usage: 478.6+ KB


In [33]:
df.drop(['VehicleID'], axis = 1, inplace = True)

In [36]:
num_col = ['Year', 'Amount (Million Naira)', 'Distance']
# cat_col = df.select_dtypes(include = ['object', 'category']).columns
cat_col = [col for col in df.columns if col not in num_col]
cat_col

['Location', 'Maker', 'Model', 'Colour', 'Type']

In [38]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
for col in cat_col:
    df[col] = le.fit_transform(df[col])

df.head()

,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,0,17,117,2011,16,2.2,2,80830.0
1,1,19,1049,2012,16,3.5,2,125000.0
2,2,29,908,2010,15,9.2,1,110852.0
3,0,34,508,2017,2,22.8,1,30000.0
4,1,52,569,2002,15,2.6,2,125206.0
